In [1]:
import pandas as pd
import math
import json

In [2]:
EU_AIRPORT_2019_PASSANGER_STAT_FILE_LOC = "../data/eu_airport_2019_passenger_data.csv"
EU_AIRPORT_2019_CLEANED_PASSANGER_STAT_FILE_LOC = "../data/cleaned/eu_cleaned_airports_2019_passenger_data.csv"
EU_AIRPORT_2019_CLEANED_PASSANGER_STAT_FILE_LOC_JSON = "../data/cleaned/eu_cleaned_airports_2019_passenger_data.json"

# Read Data

In [3]:
df_eu_airport_2019_passanger_stats = pd.read_csv(EU_AIRPORT_2019_PASSANGER_STAT_FILE_LOC, 
#                                         index_col = 0, 
                                                header=0)
df_eu_airport_2019_passanger_stats.head()

,REP_AIRP,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,ANTWERPEN/DEURNE airport,"14,571","14,072","16,344","23,265","26,280","29,455","31,328","31,946","33,019","30,499","19,281","19,710"
1,BRUSSELS airport,"16,68,418","16,44,664","20,02,518","22,92,161","22,82,737","24,16,112","27,38,116","26,48,822","24,94,996","23,21,743","19,97,468","19,09,758"
2,CHARLEROI/BRUSSELS SOUTH airport,"5,55,020","5,12,545","6,01,742","7,54,413","7,48,005","7,56,096","8,06,898","8,08,411","7,36,376","7,27,990","5,76,092","6,13,060"
3,LIEGE airport,"7,348","6,135","7,958","11,422","13,350","18,038","24,476","6,135","21,059","13,441","8,530","8,765"
4,OOSTENDE/BRUGGE airport,"18,717","18,369","23,924","39,641","40,148","46,636","60,451","66,842","50,790","39,455","21,189","20,737"


# Cleaning Airport Passenger Stats

In [4]:
df_eu_airport_2019_passanger_stats['REP_AIRP'] = df_eu_airport_2019_passanger_stats['REP_AIRP'].str.capitalize()

In [5]:
df_eu_airport_2019_passanger_stats['REP_AIRP'] = df_eu_airport_2019_passanger_stats['REP_AIRP'].str.replace("international ", "")

In [6]:
df_eu_airport_2019_passanger_stats['REP_AIRP'] = df_eu_airport_2019_passanger_stats['REP_AIRP'].str.replace("-", " ")

In [7]:
for month in range(1, 12 + 1):
    date = '2019-0'+str(month) if month <= 9 else '2019-' + str(month)
    df_eu_airport_2019_passanger_stats.rename(columns = {date :month}, inplace = True)
    df_eu_airport_2019_passanger_stats[month] = df_eu_airport_2019_passanger_stats[month].str.replace(",","")
    df_eu_airport_2019_passanger_stats[month] = df_eu_airport_2019_passanger_stats[month].str.replace(":","0")
    df_eu_airport_2019_passanger_stats[month] = df_eu_airport_2019_passanger_stats[month].astype(int)

In [8]:
df_eu_airport_2019_passanger_stats.set_index("REP_AIRP", inplace=True)

In [9]:
df_eu_airport_2019_passanger_stats

,1,2,3,4,5,6,7,8,9,10,11,12
REP_AIRP,,,,,,,,,,,,
Antwerpen/deurne airport,14571,14072,16344,23265,26280,29455,31328,31946,33019,30499,19281,19710
Brussels airport,1668418,1644664,2002518,2292161,2282737,2416112,2738116,2648822,2494996,2321743,1997468,1909758
Charleroi/brussels south airport,555020,512545,601742,754413,748005,756096,806898,808411,736376,727990,576092,613060
Liege airport,7348,6135,7958,11422,13350,18038,24476,6135,21059,13441,8530,8765
Oostende/brugge airport,18717,18369,23924,39641,40148,46636,60451,66842,50790,39455,21189,20737
...,...,...,...,...,...,...,...,...,...,...,...,...
Mugla/milas bodrum airport,96511,88648,102254,211327,389572,654471,782009,830869,649526,372947,100359,82060
Gazipasa alanya airport,33625,31413,35680,69239,113781,143888,169186,164279,141653,111411,41162,30605
Samsun/carsamba airport,138303,122570,135027,114797,111813,117811,136115,138535,117963,115005,123236,122597


# Create JSON Data

In [10]:
eu_airport_2019_passanger_stats_json = {}

eu_airport_2019_passanger_stats_count = len(df_eu_airport_2019_passanger_stats)

for airport_index in range(eu_airport_2019_passanger_stats_count):
    airport_monthly_data = df_eu_airport_2019_passanger_stats.iloc[airport_index]
    airport_name = airport_monthly_data.name
    monthly_data = {}
    eu_airport_2019_passanger_stats_json[airport_name] = {}
    for month in range(1, 12 +1):
        monthly_data[month] = math.floor(airport_monthly_data[month]/30)
    eu_airport_2019_passanger_stats_json[airport_name] = monthly_data
    
# eu_airport_2019_passanger_stats_json

# Save Airport Passenger Stats

In [11]:
df_eu_airport_2019_passanger_stats.to_csv(EU_AIRPORT_2019_CLEANED_PASSANGER_STAT_FILE_LOC)

In [12]:
with open(EU_AIRPORT_2019_CLEANED_PASSANGER_STAT_FILE_LOC_JSON, 'w') as f:
    json.dump(eu_airport_2019_passanger_stats_json, f)